In [47]:
import pandas
from create_train_test_val_maps import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
s_to_date = lambda x: pandas.to_datetime(x, format='%m/%d/%Y %H:%M:%S')
snapshots = pandas.read_csv('/home/cs231n/data/selected_snapshots_full_deduped_backfilled_whitened.csv', converters={'Event DateTime':s_to_date})

s_to_date = lambda x: pandas.to_datetime(x, format='%Y/%m/%d %H:%M:%S')
ok_snapshots = pandas.read_csv('/home/cs231n/data/selected_snapshots_ok_deduped_backfilled_whitened.csv', converters={'Event DateTime':s_to_date})

In [65]:
ata6_train_repairs = pandas.read_csv('ATA6_train.csv')
ata6_val_repairs = pandas.read_csv('ATA6_val.csv')
ata6_test_repairs = pandas.read_csv('ATA6_test.csv')

category_var = ['Veh Ref ID','Event Type Description','Brake Switch','Clutch Switch','Cruise Status','Dpf Regen Inhibit Sw', 
                'Dpf Thermal Mngmnt','Eng Coolant Level','DTCID']
for cat in category_var:
    snapshots[cat] = snapshots[cat].astype('category').cat.codes
    ok_snapshots[cat] = ok_snapshots[cat].astype('category').cat.codes

try:
    snapshots = snapshots.drop(['DTC'],1)
except:
    print('')
try:
    snapshots = snapshots.drop(['Unnamed: 0'],1)
except:
    print('')
try:
    snapshots = snapshots.drop(['Ignition Cycle Counter'],1)
except:
    print('')
    
try:
    ok_snapshots = ok_snapshots.drop(['DTC'],1)
except:
    print('')
try:
    ok_snapshots = ok_snapshots.drop(['Unnamed: 0'],1)
except:
    print('')
try:
    ok_snapshots = ok_snapshots.drop(['Ignition Cycle Counter'],1)
except:
    print('')

train_veh_ids = ata6_train_repairs['Chassis Reference Number'].unique().astype(int)
val_veh_ids = ata6_val_repairs['Chassis Reference Number'].unique().astype(int)
test_veh_ids = ata6_test_repairs['Chassis Reference Number'].unique().astype(int)

all_veh_ids = ok_snapshots['Veh Ref ID'].unique().astype(int)
train_ok_ids = all_veh_ids[0:100]
val_ok_ids = all_veh_ids[100:120]
test_ok_ids = all_veh_ids[120:144]

ids_tuple = (train_veh_ids, val_veh_ids, test_veh_ids)
ids_ok_tuple = (train_ok_ids, val_ok_ids, test_ok_ids)
repairs_tuple = (ata6_train_repairs, ata6_val_repairs, ata6_test_repairs)

In [66]:
create_maps(10, 10, snapshots, ok_snapshots, ids_tuple, ids_ok_tuple, repairs_tuple)

In [72]:
for nwin in range(10,100,10):
    for wsize in range(10,100,10):
        print("Creating num windows {} with size {}".format(nwin, wsize))
        create_maps(nwin, wsize, snapshots, ok_snapshots, ids_tuple, ids_ok_tuple, repairs_tuple)

Creating num windows 10 with size 10
Creating num windows 10 with size 20
Creating num windows 10 with size 30
Creating num windows 10 with size 40
Creating num windows 10 with size 50
Creating num windows 10 with size 60
Creating num windows 10 with size 70
Creating num windows 10 with size 80
Creating num windows 10 with size 90
Creating num windows 20 with size 10
Creating num windows 20 with size 20
Creating num windows 20 with size 30
Creating num windows 20 with size 40
Creating num windows 20 with size 50
Creating num windows 20 with size 60
Creating num windows 20 with size 70
Creating num windows 20 with size 80
Creating num windows 20 with size 90
Creating num windows 30 with size 10
Creating num windows 30 with size 20
Creating num windows 30 with size 30
Creating num windows 30 with size 40
Creating num windows 30 with size 50
Creating num windows 30 with size 60
Creating num windows 30 with size 70
Creating num windows 30 with size 80
Creating num windows 30 with size 90
C

In [67]:
train_combined, val_combined, test_combined = load_maps(10, 10)

In [68]:
train_combined.keys()

dict_keys([0, 1, 2, 3, 4, 517, 6, 7, 520, 9, 10, 11, 12, 13, 14, 15, 16, 529, 530, 19, 20, 21, 22, 535, 24, 537, 26, 539, 540, 427, 30, 31, 544, 33, 18, 35, 36, 37, 38, 39, 40, 41, 42, 43, 556, 45, 558, 559, 560, 8, 50, 51, 564, 565, 54, 567, 568, 57, 58, 571, 60, 61, 62, 63, 64, 65, 66, 523, 580, 69, 70, 583, 584, 73, 586, 75, 76, 77, 78, 79, 80, 81, 514, 598, 88, 72, 602, 91, 92, 93, 606, 101, 608, 97, 610, 612, 613, 102, 17, 616, 617, 106, 619, 108, 621, 623, 112, 113, 114, 627, 628, 118, 105, 120, 633, 122, 124, 618, 126, 127, 128, 129, 130, 131, 132, 645, 134, 136, 137, 138, 23, 140, 141, 142, 27, 90, 150, 25, 111, 5, 160, 161, 163, 165, 166, 168, 28, 170, 174, 29, 177, 178, 179, 180, 542, 183, 184, 116, 187, 189, 190, 191, 192, 193, 203, 200, 631, 34, 206, 209, 212, 213, 121, 218, 220, 222, 604, 225, 226, 227, 230, 231, 232, 235, 236, 237, 242, 56, 247, 251, 252, 253, 254, 255, 257, 258, 259, 260, 44, 268, 277, 281, 47, 284, 285, 95, 48, 49, 297, 298, 303, 304, 563, 310, 52, 53, 

In [69]:
len(list(train_combined[232][43007][0][0]))

61

In [70]:
len(list(train_combined[115][0][0][0]))

61